In [1]:
from tensorflow import keras
from tensorflow.keras.datasets import cifar10 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

In [2]:
# input image dimensions(pixel)
img_rows, img_cols = 32, 32
batch_size = 2400
num_classes = 10

#訓練次數
epochs = 100

In [3]:
# x為圖片的陣列，y為答案陣列
#例如：x_train有60000筆28×28的data。
#例如：x_test有10000筆28×28的data。
(x_train, y_train),(x_test, y_test) = cifar10.load_data() 

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#特徵值進行標準化，方法是除以255（因為圖像的像素點介於0~255之間），可讓所有的特徵值介於0與1之間
x_train /= 255 
x_test /= 255

# CNN 輸出層的每筆 Data 都是一維陣列
# 需將 Train Label(答案) 資料轉換為 One-Hot Encoding, 也就是
# 1 轉換為 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
# 5 轉換為 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
# 9 轉換為 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

input_shape=(32,32,3)

In [4]:
#建立一個線性堆疊模型，之後我們只要將建立的神經網路層依次加入即可。
model = Sequential()

#建立卷積層並加入model
model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))

#輸出層有 10 個神經元 (因為數字只有 0 ~ 9)
model.add(Dense(num_classes, activation='softmax'))

#顯示model資訊
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          3

In [5]:
# 設定模型的訓練方式
model.compile(loss=keras.losses.categorical_crossentropy,  # 設定 Loss 損失函數
              optimizer=keras.optimizers.Adam(0.001),      # 設定 Optimizer 最佳化方法 
              metrics=['accuracy'])                        # 設定 Model 評估準確率方法 為 accuracy

# 開始訓練
# CNN 輸入層的每筆 Data 都是三維陣列, 這就是為何 x 輸入 標準化_x_train 或 原始_x_train 皆可
# CNN 輸出層的每筆 Data 都是一維陣列, 這就是為何 y 輸入 經過前處理_y_train, 而非 原始train_label
# (若準確率不高, 改善 1: 可再執行這個函數, 進行重覆訓練)
# (若準確率不高, 改善 2: 重新建立 Model, 且增加 卷積層 filter 數, 重新進行訓練)
# (若準確率不高, 改善 3: 重新建立 Model, 且增加 隱藏層 units 神經元數, 重新進行訓練)
# (若準確率不高, 改善 4: 重新建立 Model, 且增加 隱藏層 layer, 重新進行訓練)
# (若準確率不高, 改善 5: 重新建立 Model, 且更換訓練方式（神經網路）, 重新進行訓練)

model.fit(x_train,                          # 設定 圖片 Features 特徵值
          y_train,                          # 設定 圖片 Label    答案
          batch_size=batch_size,            # 設定 訓練時每批次有多少筆 (值100以上, 值越大訓練速度越快, 但需記憶體要夠大)
          epochs=epochs,                    # 設定 訓練次數 (值越大訓練時間越久, 但訓練越精準)
          verbose=2,                        # 是否 顯示訓練過程 (0: 不顯示, 1: 詳細顯示, 2: 簡易顯示)
          validation_data=(x_test, y_test)) # 在每輪結束時用(x_test, y_test)做測試

#使用未參與訓練的test dataset測試
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
21/21 - 6s - loss: 2.1841 - accuracy: 0.1790 - val_loss: 2.1215 - val_accuracy: 0.2633
Epoch 2/100
21/21 - 2s - loss: 1.9199 - accuracy: 0.3001 - val_loss: 1.8374 - val_accuracy: 0.3787
Epoch 3/100
21/21 - 2s - loss: 1.7136 - accuracy: 0.3745 - val_loss: 1.6954 - val_accuracy: 0.4231
Epoch 4/100
21/21 - 2s - loss: 1.6105 - accuracy: 0.4122 - val_loss: 1.6049 - val_accuracy: 0.4594
Epoch 5/100
21/21 - 2s - loss: 1.5099 - accuracy: 0.4508 - val_loss: 1.5305 - val_accuracy: 0.4904
Epoch 6/100
21/21 - 2s - loss: 1.4427 - accuracy: 0.4719 - val_loss: 1.4946 - val_accuracy: 0.5168
Epoch 7/100
21/21 - 2s - loss: 1.3924 - accuracy: 0.4919 - val_loss: 1.4513 - val_accuracy: 0.5255
Epoch 8/100
21/21 - 2s - loss: 1.3557 - accuracy: 0.5103 - val_loss: 1.4153 - val_accuracy: 0.5542
Epoch 9/100
21/21 - 2s - loss: 1.3138 - accuracy: 0.5259 - val_loss: 1.3711 - val_accuracy: 0.5672
Epoch 10/100
21/21 - 2s - loss: 1.2582 - accuracy: 0.5471 - val_loss: 1.2979 - val_accuracy: 0.5875
Epoch 11/

21/21 - 2s - loss: 0.5256 - accuracy: 0.8129 - val_loss: 0.6810 - val_accuracy: 0.7753
Epoch 84/100
21/21 - 2s - loss: 0.5225 - accuracy: 0.8135 - val_loss: 0.6668 - val_accuracy: 0.7761
Epoch 85/100
21/21 - 2s - loss: 0.5200 - accuracy: 0.8140 - val_loss: 0.6652 - val_accuracy: 0.7760
Epoch 86/100
21/21 - 2s - loss: 0.5159 - accuracy: 0.8166 - val_loss: 0.6542 - val_accuracy: 0.7829
Epoch 87/100
21/21 - 2s - loss: 0.5127 - accuracy: 0.8154 - val_loss: 0.6747 - val_accuracy: 0.7694
Epoch 88/100
21/21 - 2s - loss: 0.5155 - accuracy: 0.8150 - val_loss: 0.6688 - val_accuracy: 0.7782
Epoch 89/100
21/21 - 2s - loss: 0.5169 - accuracy: 0.8175 - val_loss: 0.6544 - val_accuracy: 0.7810
Epoch 90/100
21/21 - 2s - loss: 0.5035 - accuracy: 0.8206 - val_loss: 0.6604 - val_accuracy: 0.7829
Epoch 91/100
21/21 - 2s - loss: 0.5003 - accuracy: 0.8222 - val_loss: 0.6852 - val_accuracy: 0.7673
Epoch 92/100
21/21 - 2s - loss: 0.5030 - accuracy: 0.8195 - val_loss: 0.6402 - val_accuracy: 0.7850
Epoch 93/100
